In [11]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from stockfish import Stockfish
import chess
import chess.engine
import math
import random
from evaluation_pipeline_copy import generate_synthetic_data_dict

In [12]:
# data = pd.read_csv('Valgte punkter fra ensemble')
df = pd.read_csv(r"chessData.csv")

In [13]:
data = generate_synthetic_data_dict(50,1,1,df)
data

['g7g6',
 'd1f3',
 'd8h4',
 'e1e2',
 'e8d7',
 'f1e2',
 'f8a3',
 'd1g4',
 'h8g8',
 'd1g4',
 'f8e7',
 'f1e2',
 'd8b6',
 'f1a6',
 'c6b4',
 'd1c2',
 'c6a5',
 'c1d2',
 'c6a5',
 'd1a4',
 'c6d8',
 'a1c1',
 'b4a5',
 'e2c3',
 'f8f7',
 'd2c3',
 'c6b8',
 'b2b3',
 'b6d4',
 'h1g1',
 'b6a6',
 'd2b4',
 'b7b5',
 'd2h6',
 'g8f8',
 'c2d3',
 'a8b8',
 'h7g8',
 'e5d3',
 'c3b4',
 'g7g6',
 'c2f5',
 'c4a3',
 'c2c4',
 'g7g5',
 'c2c4',
 'c8a6',
 'e2e6',
 'f6d4',
 'd3e4']

In [14]:
path=("stockfish-windows-x86-64\stockfish\stockfish-windows-x86-64.exe")

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\nunni\AppData\Local\Temp\ipykernel_12164\315037162.py:1: SyntaxWarning: invalid escape sequence '\s'
  path=("stockfish-windows-x86-64\stockfish\stockfish-windows-x86-64.exe")


In [15]:
fen=("rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2")

In [16]:
def stockfish_evaluation(board, time_limit = 0.01):
    engine = chess.engine.SimpleEngine.popen_uci(path)
    result = engine.analyse(board, chess.engine.Limit(time=time_limit))
    return result['score']

def evaluate_move(stockfish,fen,move):
    board = chess.Board(fen)
    board.push_uci(move)
    return stockfish_evaluation(board)

board = chess.Board(fen)
result = stockfish_evaluation(board)

stockfish = Stockfish(path)

In [17]:
score = []

for i in range(len(data)):
    move = data[i]
    fen = df['FEN'][i]
    if 'w' in fen:
        end = evaluate_move(stockfish, fen, move).white().score()
        if type(end) == type(None):
            score.append(0)
        else:
            score.append(end - int(df['Evaluation'][i]))
    else:
        end = evaluate_move(stockfish, fen, move).black().score()
        if type(end) == type(None):
            score.append(0)
        else:
            score.append(end - int(df['Evaluation'][i]))
        
print(score)

[-43, -94, -149, -243, -117, -147, -545, -66, -171, -120, -103, -205, -60, -550, -136, -122, -203, 27, -627, -402, -291, -645, -435, -506, -527, -66, -503, -291, -593, -519, -611, -404, -408, -287, -408, -754, -443, -449, -713, -590, -638, -982, -523, -611, -705, -1075, -692, 0, -1009, -642]


In [18]:
def evaluate_list_white(stockfish,fen,list):
    res = []
    for move in list:
        res.append(evaluate_move(stockfish,fen,move).white().score())
    return res

def evaluate_list_black(stockfish,fen,list):
    res = []
    for move in list:
        res.append(evaluate_move(stockfish,fen,move).black().score())
    return res

def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

In [19]:
mean_score = []
for i in range(len(data)):
    score = []
    move = data[i]
    fen = df['FEN'][i]
    if 'w' in fen:
        move_score = evaluate_list_white(stockfish,fen,gen_legal_moves(chess.Board(fen))),"\n",gen_legal_moves(chess.Board(fen))
        for j in range(len(move_score[0])):
            score.append(move_score[0][j] - int(df['Evaluation'][i]))
        mean_score.append(np.mean(score))
    else:
        move_score = evaluate_list_black(stockfish,fen,gen_legal_moves(chess.Board(fen))),"\n",gen_legal_moves(chess.Board(fen))
        for j in range(len(move_score[0])):
            score.append(move_score[0][j] - int(df['Evaluation'][i]))
        mean_score.append(np.mean(score))
    
        


OSError: [WinError 1455] Sidefilen er for lille til, at denne handling kan gennemføres

In [ ]:
print(mean_score)

[-81.65, -96.73333333333333, -155.46666666666667]
